In [562]:
import numpy as np
import pandas as pd

SPLIT_COEF = .2       # .15
RANDOM_STATE = 42      # 10
N_JOBS = 12
CV_NUM = 5
DATASET_NAME = "BodyFat"

In [563]:
# df_train = pd.read_csv('train_Frankes_f.txt', header=None)
# df_test = pd.read_csv('test_Frankes_f.txt', header=None)

# df_train.head()

In [564]:
# train_X = df_train.iloc[:, :-1]
# train_y = df_train.iloc[:,-1]
#
# test_X = df_test.iloc[:, :-1]
# test_y = df_test.iloc[:,-1]

In [565]:
df = pd.read_csv('../bodyfat.csv')
df.drop(columns="Density", inplace=True)
df.head()

,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,12.3,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,6.1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,25.3,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,10.4,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,28.7,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [566]:
y = df.BodyFat
X = df.drop(columns="BodyFat").to_numpy()
X
# X = df.iloc[:, :-1]
# y = df.iloc[:,-1]

array([[ 23.  , 154.25,  67.75, ...,  32.  ,  27.4 ,  17.1 ],
       [ 22.  , 173.25,  72.25, ...,  30.5 ,  28.9 ,  18.2 ],
       [ 22.  , 154.  ,  66.25, ...,  28.8 ,  25.2 ,  16.6 ],
       ...,
       [ 72.  , 186.75,  66.  , ...,  31.3 ,  27.2 ,  18.  ],
       [ 72.  , 190.75,  70.5 , ...,  30.5 ,  29.4 ,  19.8 ],
       [ 74.  , 207.5 ,  70.  , ...,  33.7 ,  30.  ,  20.9 ]])

In [567]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(
        X, y, test_size=.2)
train_X.shape, test_X.shape, test_y.shape

((201, 13), (51, 13), (51,))

In [568]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler

scaler = MaxAbsScaler()
# scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

# SGTM + GRNN results

In [569]:
from src.method_1 import SGTM_GRNN
from src.GRNN_ import GRNN
from src.SGTM import GTM

In [570]:
from sklearn.model_selection import GridSearchCV

def find_sigmas(X, y, params: dict, scoring="neg_mean_absolute_error") -> dict:
    gs = GridSearchCV(
            SGTM_GRNN(),
            params,
            n_jobs=N_JOBS,
            scoring=scoring,
            cv=CV_NUM
    )
    gs.fit(X, y)
    print(gs.best_score_, gs.best_params_)
    return  gs.best_params_


In [571]:
params = {"sigma": np.arange(.001, 10.001, .001)}

# best_params_1 = find_sigmas(train_X, train_y, params=params)

In [572]:
# model_1 = SGTM_GRNN(sigma=best_params_1["sigma"])
model_1 = SGTM_GRNN(sigma=0.09)
model_1.fit(train_X, train_y)
model_1_pred = model_1.predict(test_X)

### on test data

In [573]:
from  src.errors_to_excel import ErrorsToExcel

errors = ErrorsToExcel(f"results/m1_errors_{DATASET_NAME}")
errors.add_errors_measurment(
        "method_1(sgtm + grnn) on test data",
        test_y,
        model_1_pred
)

{
    "method_1(sgtm + grnn) on test data": {
        "MaxE": 8.223628369349541,
        "MAE": 2.9259330560380743,
        "MSE": 13.67672389299918,
        "RMSE": 3.69820549631834,
        "MAPE": 0.17657131527869446,
        "MedianAE": 2.6608917328648474,
        "R2_score": 0.7865471041827
    }
}


### on train data

In [574]:
model_1_train_pred = model_1.predict(train_X)
errors.add_errors_measurment(
        "method_1(sgtm + grnn) on train data",
        train_y,
        model_1_train_pred
)

{
    "method_1(sgtm + grnn) on train data": {
        "MaxE": 9.763016069519264,
        "MAE": 3.0344192054984207,
        "MSE": 14.231520871136103,
        "RMSE": 3.7724688032025,
        "MAPE": 38006590959882.56,
        "MedianAE": 2.667387053373977,
        "R2_score": 0.7999279342652023
    }
}


# SGTM results

In [575]:
gtm = GTM(numb_of_steps=train_X.shape[1])
gtm.fit(train_X, train_y)

GTM(numb_of_steps=13)

### on test data

In [576]:
gtm_test_pred = gtm.predict(test_X)
errors.add_errors_measurment(
        "sgtm on test data",
        test_y,
        gtm_test_pred
)

{
    "sgtm on test data": {
        "MaxE": 8.940629648473998,
        "MAE": 3.0426936610330024,
        "MSE": 14.251356171560147,
        "RMSE": 3.7750968426730656,
        "MAPE": 0.1874032400089867,
        "MedianAE": 2.5683081030146226,
        "R2_score": 0.7775788070343078
    }
}


### on train data

In [577]:
gtm_train_pred = gtm.predict(train_X)
errors.add_errors_measurment(
        "sgtm on train data",
        train_y,
        gtm_train_pred
)

{
    "sgtm on train data": {
        "MaxE": 11.231233170578268,
        "MAE": 3.5512502795732854,
        "MSE": 18.552589701910392,
        "RMSE": 4.30727172371449,
        "MAPE": 117573719035494.77,
        "MedianAE": 3.1139686806181963,
        "R2_score": 0.7391807256580982
    }
}


# GRNN results

In [578]:
params = {"sigma": np.arange(.001, 10.001, .001)}

best_params_2 = find_sigmas(train_X, train_y, params=params)

-3.839513754990847 {'sigma': 0.079}


In [579]:
grnn = GRNN(sigma=best_params_2["sigma"])
grnn.fit(train_X, train_y)

GRNN(sigma=0.079)

In [580]:
grnn_test_pred = grnn.predict(test_X)
errors.add_errors_measurment(
        "grnn on test data",
        test_y,
        grnn_test_pred
)

{
    "grnn on test data": {
        "MaxE": 15.324280628413703,
        "MAE": 3.686482167329282,
        "MSE": 21.265118529568294,
        "RMSE": 4.611411771851251,
        "MAPE": 0.2299006137593045,
        "MedianAE": 2.7454390783567213,
        "R2_score": 0.6681148814916162
    }
}


# Polynomial features extending

In [581]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
train_X_ext = poly.fit_transform(train_X)
test_X_ext = poly.fit_transform(test_X)

## SGTM + GRNN results

In [582]:
best_params_3 = find_sigmas(train_X, train_y, params=params)

-3.839513754990847 {'sigma': 0.079}


In [583]:
model_2 = SGTM_GRNN(sigma=best_params_3["sigma"])
model_2.fit(train_X_ext, train_y)
model_2_pred = model_2.predict(test_X_ext)

# on test data

In [584]:
errors.add_errors_measurment(
        "method_1_v2(sgtm+grnn) on test data with expended features",
        test_y,
        model_2_pred
)

{
    "method_1_v2(sgtm+grnn) on test data with expended features": {
        "MaxE": 16.771795711183586,
        "MAE": 5.135622576774993,
        "MSE": 40.434652681783135,
        "RMSE": 6.358824787787688,
        "MAPE": 0.3143722693608948,
        "MedianAE": 4.224588048278978,
        "R2_score": 0.36893558912077373
    }
}


### on train data

In [585]:
model_2_train_pred = model_2.predict(train_X_ext)
errors.add_errors_measurment(
        "method_1_v2(sgtm+grnn) on train data with expended features",
        train_y,
        model_2_train_pred
)

{
    "method_1_v2(sgtm+grnn) on train data with expended features": {
        "MaxE": 1.6333954438493805,
        "MAE": 0.08509208585057222,
        "MSE": 0.06255102842688559,
        "RMSE": 0.2501020360310679,
        "MAPE": 30678767591.055946,
        "MedianAE": 0.005173862570625687,
        "R2_score": 0.9991206341483443
    }
}


# SGTM results

In [586]:
gtm = GTM(numb_of_steps=train_X_ext.shape[1])
gtm.fit(train_X_ext, train_y)

GTM(numb_of_steps=105)

### on test data

In [587]:
gtm_test_pred = gtm.predict(test_X_ext)
errors.add_errors_measurment(
        "sgtm on test data with expended features",
        test_y,
        gtm_test_pred
)

{
    "sgtm on test data with expended features": {
        "MaxE": 13.580665146920676,
        "MAE": 4.991669274694409,
        "MSE": 34.30837472284131,
        "RMSE": 5.857335121268144,
        "MAPE": 0.31231328254939744,
        "MedianAE": 3.746544443435784,
        "R2_score": 0.46454852838521266
    }
}


### on train data

In [588]:
gtm_train_pred = gtm.predict(train_X_ext)
errors.add_errors_measurment(
        "sgtm on train data with expended features",
        train_y,
        gtm_train_pred
)

{
    "sgtm on train data with expended features": {
        "MaxE": 8.232179279393838,
        "MAE": 2.2285309926493624,
        "MSE": 8.328457940406825,
        "RMSE": 2.885906779576711,
        "MAPE": 37060414466547.59,
        "MedianAE": 1.8562893239968083,
        "R2_score": 0.8829154101230257
    }
}


In [589]:
best_params_4 = find_sigmas(train_X, train_y, params=params)

-3.839513754990847 {'sigma': 0.079}


In [590]:
grnn = GRNN(sigma=best_params_4["sigma"])
grnn.fit(train_X_ext, train_y)

GRNN(sigma=0.079)

In [591]:
grnn_test_pred = grnn.predict(test_X_ext)
errors.add_errors_measurment(
        "grnn on test data with expended features",
        test_y,
        grnn_test_pred
)

{
    "grnn on test data with expended features": {
        "MaxE": 15.246774403310859,
        "MAE": 4.266170587747535,
        "MSE": 27.58775924738202,
        "RMSE": 5.252405091706277,
        "MAPE": 0.2562991927042519,
        "MedianAE": 3.026193253224342,
        "R2_score": 0.5694373048301077
    }
}


In [592]:
errors.write()